# Notebook 4 – Transfer Learning in NeuralLib

In this notebook we focus on **transfer learning (TL)** with NeuLib:

- Start from a pretrained ECG **ProductionModel**.
- Build a **TransferLearningModel (TLModel)** that reuses part of the pretrained GRU encoder.
- Add extra layers for the new task/domain.
- Inject pretrained weights, set up freezing/unfreezing, and fine-tune on new data.

#### How exactly does NeuralLib work?
<img src="tutorial_plots/NeuralLibframework.png">

## Step 0 – Environment setup

For this tutorial NeuLib is imported from a local clone of the repository.
We temporarily add the project directory to `sys.path` so that Python can find the package.


In [ ]:
# Cell to be removed once the package is stable
import sys
import os

# Get the absolute path of your project directory
project_path = os.path.abspath("..")
print(project_path)

# Add the project directory to sys.path
if project_path not in sys.path:
    sys.path.append(project_path)

### Import NeuLib modules for transfer learning

We import:

- The GRU-based architecture `GRUseq2seq`.
- Dataset paths for ECG peak detection.
- Transfer learning helpers from `model_hub`:
  - `TLModel`, `TLFactory`
  - `list_production_models`
  - `build_tl_arch_config`, `build_pretrained_layer_map`, `build_freeze_phase`


In [ ]:
from NeuralLib.architectures import GRUseq2seq
from NeuralLib.config import DATASETS_ECG_G # peak detection dataset
from NeuralLib.model_hub import (
    TLModel, TLFactory, 
    list_production_models, 
    build_pretrained_layer_map, 
    build_tl_arch_config,
    build_freeze_phase
    )

### Step 1 – Load target-domain data

We first define the paths for the **target** ECG dataset:

- `X`: input ECG signals.
- `Y_BIN`: binary labels for peak detection.

This dataset represents the **target domain** on which we will fine-tune the model.


In [ ]:
X = os.path.join(DATASETS_ECG_G, "sample", "x")
Y_BIN = os.path.join(DATASETS_ECG_G, "sample", "y_bin")

## Step 2 – Load a pretrained ProductionModel

NeuLib provides **ProductionModels** stored on the model hub.
We use a `TLFactory` to:

- List available pretrained models.
- Download or load cached weights.
- Prepare them for transfer learning.


In [ ]:
factory = TLFactory()

# List available production models
list_production_models()

### Select a pretrained ECGPeakDetector

We now load the `ECGPeakDetector` ProductionModel.

This gives us access to:

- The pretrained GRU-based encoder–decoder.
- Its architecture configuration (hidden sizes, bidirectionality, etc.).
- The checkpoint files stored locally in the NeuLib cache.

In [ ]:
# Load Production Model
factory.load_production_model(model_name="ECGPeakDetector")
prod_model = factory.models["ECGPeakDetector"]

print("\nPretrained GRU model (encoder):")
print(prod_model.model)

### Step 3 – Build a TransferLearningModel configuration

We now decide **how much** of the pretrained model to reuse and
**how** to extend it for the new task.

Key design choices:

- Number of GRU layers to reuse from the pretrained model (`reuse_n_gru_layers`).
- Additional hidden dimensions (`extra_hid_dims`) appended on top.
- Bidirectionality per layer (`bidir_per_layer`).
- New task type (e.g. regression vs classification).

#### 3.1 – Define TL architecture parameters

`build_tl_arch_config`:

- Reads the configuration of the pretrained `prod_model`.
- Reuses the first `reuse_n_gru_layers` layers.
- Appends new GRU layers with `extra_hid_dims`.
- Builds a configuration dictionary `arch_params` for the TLModel.
- Returns `reuse_n` (the actual number of reused layers).

In [ ]:
arch_params, reuse_n = build_tl_arch_config(
    prod_model=prod_model,
    reuse_n_gru_layers=2,
    extra_hid_dims=[128, 128],
    bidir_per_layer=[True, True, True, False],
    learning_rate=1e-3,
    model_name_suffix="_TL_PeakDetector",
    task="regression",
    num_classes=1,
    multi_label=False,    # BCEWithLogits for binary peaks
    fc_out_bool=False,
)

<img src="tutorial_plots/FrozenLayer.png">

In [ ]:
tl_model = TLModel(GRUseq2seq, **arch_params)
print(f"TL_model structure{tl_model}")

## Step 4 – Map pretrained layers and define freezing strategy

To perform TL we must:

1. Decide which GRU layers reuse pretrained weights.
2. Build a **mapping** between layers in the ProductionModel and the TLModel.
3. Choose a **freezing strategy** (which layers to freeze/unfreeze during fine-tuning).

### 4.1 – Build layer mapping

`build_pretrained_layer_map`:

- Reads the GRU layers from the pretrained `prod_model`.
- Matches them to the first `reuse_n` GRU layers of `tl_model`.
- Returns a dictionary that will be used to inject weights.


In [ ]:
# Build layer mapping from pretrained model
layer_mapping = build_pretrained_layer_map(prod_model, reuse_n)

### 4.2 – Decide which layers to freeze

We use `build_freeze_phase` to:

- Freeze reused layers during the first phase of fine-tuning.
- Keep newly added layers trainable.

Here we use the **`new_only`** strategy:

- Reused GRU layers are frozen.
- New GRU layers remain unfrozen.

In [ ]:
# Automatic freeze layer settings
freeze_layers, unfreeze_layers = build_freeze_phase(
    tl_model,
    reuse_n,
    strategy="new_only",
    )

<img src="tutorial_plots/ExtraHiddenLayers.png">

## Step 5 – Inject weights and configure the TLModel

We are now ready to:

- Inject pretrained weights into selected GRU layers of `tl_model`.
- Freeze and unfreeze layers according to our strategy.
- Finalize the TLModel configuration.


In [ ]:
# Configure a TransferLearningModel by injecting weights and managing layer freezing/unfreezing
factory.configure_tl_model(
    tl_model=tl_model,
    layer_mapping=layer_mapping,
    freeze_layers=freeze_layers,
    unfreeze_layers=unfreeze_layers,
)

## Step 6 – Fine-tune the TLModel

We now define training parameters for fine-tuning on the target dataset:

- Number of epochs and batch size.
- Early stopping patience.
- Dataset name and description of the TL objective.
- Device and logging options.


In [ ]:
# Configure training parameters
train_params = {
    "path_x": X,
    "path_y": Y_BIN,
    "epochs": 300,
    "batch_size": 32,
    "patience": 20,
    "dataset_name": "private_gib01",
    "trained_for": "fine-tuning peak detection",
    "all_samples": True,
    "samples": None,
    "gpu_id": 1,
    "enable_tensorboard": True,
}

### 6.1 – Start TL training

We call the TL-specific training routine on `tl_model`.

During training:

- Reused layers are kept frozen (as configured).
- New layers adapt to the target dataset.
- Training logs and checkpoints are saved under the TL experiment directory.


In [ ]:
# Start TL training
print("\n Starting TL training...")
tl_model.train_tl(**train_params)
print(" TL training finished.")

## Summary – Notebook 4

In this notebook we:

- Loaded a pretrained ECG **ProductionModel** from the NeuLib model hub.
- Built a **TransferLearningModel** that reuses part of the GRU encoder.
- Decided how many layers to reuse and how many new layers to add.
- Injected pretrained weights and set a freezing/unfreezing strategy.
- Fine-tuned the TLModel on a target-domain ECG dataset.

This shows how NeuLib supports practical **transfer learning pipelines**
for biosignal processing with minimal custom code.


In [ ]:
#TODO: Future plans

In [ ]:
from NeuralLib.architectures.upload_to_hugging import upload_production_model

# Example
token='inser_your_token'
description = "GRU-based model for ECG peak detection"
upload_production_model(local_dir=r'/Users/groupies/Documents/NOVAProjects/NeuralLib-fork/hugging_prodmodels/ECGDenoiser',
                        repo_name='Nianfei/ECGDenoiserNL',
                        token = token,
                        model_name='ECGDenoiserNL',
                        description=description)